In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [2]:
#sliding window sequence builder
def create_sequences(df, feature_cols, target_col, window=20):
    X, y = [], []
    values = df[feature_cols].values
    targets = df[target_col].values

    for i in range(len(df) - window):
        X.append(values[i:i+window])
        y.append(targets[i+window])

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int32)


In [3]:
def build_cnn_model(n_features, window):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(window, n_features)),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [4]:
import pandas as pd
df = pd.read_csv("E://PHD/Course materials/Sem 1/Data Mining/Assignment/workspace/stock_price_trend_prediction/stock_price_pred_data_mining/Data/05_cluster_output/cluster_output_Kmeans.csv")
# 1. Compute next-week (5 trading days) forward return per stock
df['next_week_close'] = df.groupby('stock_id')['Close'].shift(-5)
df['next_week_return'] = (df['next_week_close'] - df['Close']) / df['Close']
# 2. Binary target: 1 = uptrend, 0 = downtrend or flat
df["target"] = (df["Close"].shift(-5) > df["Close"]).astype(int)
# 3. Drop rows where we can't compute future return (last 5 days per stock)
df = df.dropna(subset=['next_week_return'])
# Optional: drop helper column
df = df.drop(columns=['next_week_close'])
drop_cols = ['Date', 'stock_id', 'cluster', 'target', 'next_week_return']
feature_cols = [c for c in df.columns if c not in drop_cols]

print("Number of features:", len(feature_cols))
print("Features:", feature_cols)

Number of features: 23
Features: ['Open', 'High', 'Low', 'Close', 'Volume', 'MA7', 'MA21', 'EMA20', 'EMA50', 'RSI_14', 'MACD', 'MACD_signal', 'MACD_hist', 'BB_upper', 'BB_middle', 'BB_lower', 'CCI_14', 'CMF_20', 'Stoch_K', 'Stoch_D', 'Momentum_10', 'Daily_Return', 'Log_Return']


In [ ]:
WINDOW = 20  # typically 20, 30, or 60
cluster_cnn_models = {}
cluster_metrics = {}

for cl in df['cluster'].unique():
    print(f"\n Training 1D CNN for Cluster {cl}")

    sub = df[df['cluster'] == cl].copy().sort_values('Date')

    # ---- Build sequences
    X, y = create_sequences(sub, feature_cols, 'target', window=WINDOW)

    if len(X) < 200:
        print(f" Cluster {cl}: Not enough samples")
        continue

    # Train/Test split (time-series aware)
    split_idx = int(len(X) * 0.7)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # Build model
    model = build_cnn_model(n_features=X.shape[2], window=WINDOW)

    # Train CNN
    es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=64,
        validation_split=0.2,
        callbacks=[es],
        verbose=1
    )

    cluster_cnn_models[cl] = model

    # ---- Predictions
    y_prob = model.predict(X_test).flatten()
    y_pred = (y_prob > 0.5).astype(int)

    # ---- Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    try:
        auc = roc_auc_score(y_test, y_prob)
    except:
        auc = 0.0

    cluster_metrics[cl] = [acc, prec, rec, f1, auc]

    print(f"""
       Cluster {cl} — 1D CNN Results
       ------------------------------
       Accuracy     : {acc:.4f}
       Precision    : {prec:.4f}
       Recall       : {rec:.4f}
       Macro F1     : {f1:.4f}
       ROC-AUC      : {auc:.4f}
    """)



📌 Training 1D CNN for Cluster 1
Epoch 1/30


c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


499/499 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5109 - loss: 8933.0127 - val_accuracy: 0.5367 - val_loss: 13.4100
Epoch 2/30
499/499 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5168 - loss: 11.7614 - val_accuracy: 0.5672 - val_loss: 0.8017
Epoch 3/30
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5177 - loss: 0.9943 - val_accuracy: 0.5661 - val_loss: 0.7097
Epoch 4/30
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5176 - loss: 0.8209 - val_accuracy: 0.5668 - val_loss: 0.8337
Epoch 5/30
499/499 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5178 - loss: 3.0164 - val_accuracy: 0.5671 - val_loss: 3.2253
Epoch 6/30
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5179 - loss: 2.2499 - val_accuracy: 0.5674 - val_loss: 0.6890
Epoch 7/30
499/499 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5179 - loss: 0.6941 - val_accuracy: 0.5673 - val_loss: 0.6890
Epoch 8/30
499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5178 - loss: 0.7064 - val_accuracy: 0.5673 -

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5130 - loss: 177996.7500 - val_accuracy: 0.5790 - val_loss: 2229.5320
Epoch 2/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5200 - loss: 843.3787 - val_accuracy: 0.5758 - val_loss: 1328.4182
Epoch 3/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5204 - loss: 83.8873 - val_accuracy: 0.5754 - val_loss: 770.4920
Epoch 4/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5191 - loss: 29.6354 - val_accuracy: 0.5758 - val_loss: 630.5771
Epoch 5/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5186 - loss: 22.5449 - val_accuracy: 0.5749 - val_loss: 623.8213
Epoch 6/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5191 - loss: 39.8794 - val_accuracy: 0.5763 - val_loss: 589.4375
Epoch 7/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5199 - loss: 14.0963 - val_accuracy: 0.5758 - val_loss: 589.4545
Epoch 8/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5187 - loss: 15.5455 